# March Madness 2019 Bracket Generator Walkthrough
## By Brian McNaughton
##### Ad Majorem Dei Gloriam
Let's get this perfect bracket.

Ok, first we will load the data.
We are just using RPI rankings, Kenpom, and BPI, along with the W/L record for each team. We are going for quantity, not quality because March Madness.

In [1]:
import pandas as pd
from pprint import pprint

data_file = 'config\TeamData.csv'
teams_df = pd.read_csv(data_file)

teams_df.head()

,Team,Win,Loss,Pct,RPI,Kenpom,BPI
0,Villanova,31,3,0.911765,1,30.78,20.7
1,North Carolina,27,7,0.794118,5,28.01,19.5
2,Kansas,28,4,0.875000,3,25.98,16.8
3,Gonzaga,32,1,0.969697,8,33.05,19.6
4,Duke,27,8,0.771429,6,25.09,18.0


Sweet, the data is there. Now let's see if we can make some power rankings of our own, and see how they look.

Define your own formula here below.

In [2]:
# This formula defines the power rating for each team
def formula1(team, win, loss, pct, RPI, Kenpom, BPI):
    power = pct * (1/(40+RPI)) * (Kenpom/35) * BPI
    return power

Now run the block below to load up your formula and see the power ratings for each team.

In [3]:
# Generate power rating for each team
power_ratings = dict()
for index, team in teams_df.iterrows():
    name = team['Team']
    power = formula1(team['Team'],team['Win'],team['Loss'],team['Pct'],team['RPI'],team['Kenpom'],team['BPI'])
    power_ratings.update({name:power})

# Print the top 10 teams to check the ratings
top = 10
sorted_ratings = sorted(power_ratings.items(), reverse=True, key=lambda tup: tup[1])[:top]
pprint(sorted_ratings)    

[('Villanova', 0.4048273417206104),
 ('Gonzaga', 0.37389899001583327),
 ('North Carolina', 0.2753924369543905),
 ('Kentucky', 0.2582733573823325),
 ('Kansas', 0.25375813953488374),
 ('Louisville', 0.22171914893617017),
 ('Duke', 0.21639290138821116),
 ('Florida', 0.19905599999999998),
 ('Oregon', 0.19318197554081865),
 ('Arizona', 0.1910924369365714)]


So now we have defined a numerical value for each team as a "power rating". Let's check how it plays out in a feew scenarios below. The probability is returned as the probability that team 1 will beat team 2.

In [4]:
# Returns the probability that team1 beats team2
def prob(team1, team2):
    return team1 / (team1+team2) * 100

In [5]:
print('Villanova - Gonzaga: ' + str(prob(power_ratings['Villanova'],power_ratings['Gonzaga'])) + ' %')
print('Villanova - New Orleans: ' + str(prob(power_ratings['Villanova'],power_ratings['New Orleans'])) + ' %')
print('Villanova - Wisconsin: ' + str(prob(power_ratings['Villanova'],power_ratings['Wisconsin'])) + ' %')

Villanova - Gonzaga: 51.985829324392526 %
Villanova - New Orleans: 99.99882647161587 %
Villanova - Wisconsin: 81.71693199362981 %


Try a few matchups for yourself below if you'd like.

In [6]:
# Change the team1 and team2 variables to try out your own matchups
team1 = 'Duke'
team2 = 'Arizona'
print(team1 + ' - ' + team2 + ': ' + str(prob(power_ratings[team1],power_ratings[team2])) + ' %')

Duke - Arizona: 53.10446316371195 %


Alright, we've got power rankings for each team, let's get on to filling out a bracket!

In [7]:
# First we'll set up the empty bracket
empty_bracket_file = 'config\EmptyBracket.csv'
empty_bracket_df = pd.read_csv(empty_bracket_file,dtype='object')

# Make a copy for manipulation
bracket_df = empty_bracket_df.copy()

empty_bracket_df.head()

,Rd1,Rd2,S16,E8,FF,C,W
0,Villanova,NaN,NaN,NaN,NaN,NaN,NaN
1,Mount St. Mary's,NaN,NaN,NaN,NaN,NaN,NaN
2,Wisconsin,NaN,NaN,NaN,NaN,NaN,NaN
3,Virgina Tech,NaN,NaN,NaN,NaN,NaN,NaN
4,Virginia,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
import random

def predict_rd(rd):
    next_rd_teams = []
    for team in rd:
        team1 = team
        team2 = next(rd)
        # print(team1 + ' - ' + team2 + ': ' + str(prob(power_ratings[team1],power_ratings[team2])) + ' %')
        odds = prob(power_ratings[team1],power_ratings[team2])
        rand = random.uniform(0,100)
        if rand < odds:
            winner = team1
        else:
            winner = team2  
        next_rd_teams.append(winner)   
        # print(odds, rand, winner)
    return next_rd_teams

def predict_bracket():
    Rd1 = iter(empty_bracket_df['Rd1'].values)
    Rd2_teams = predict_rd(Rd1)
    # print(Rd2_teams)
    Rd2 = iter(Rd2_teams)
    S16_teams = predict_rd(Rd2)
    # print(S16_teams)
    S16 = iter(S16_teams)
    E8_teams = predict_rd(S16)
    # print(E8_teams)
    E8 = iter(E8_teams)
    FF_teams = predict_rd(E8)
    # print(FF_teams)
    FF = iter(FF_teams)
    C_teams = predict_rd(FF)
    # print(C_teams)
    C = iter(C_teams)
    W = predict_rd(C)
    winner = ''.join(W)
    # print(winner)
    return Rd2_teams, S16_teams, E8_teams, FF_teams, C_teams, winner

# Now we will fill out the bracket, preparing it for writing to csv
def fill_bracket(bracket_df):
    bracket = predict_bracket()
    
    Rd2 = bracket[0]
    S16 = bracket[1]
    E8 = bracket[2]
    FF = bracket[3]
    C = bracket[4]
    winner = bracket[5]

    i = 0
    for team in Rd2:
        bracket_df.at[i,'Rd2'] = team
        i+=2
    i = 0
    for team in S16:
        bracket_df.at[i,'S16'] = team
        i+=4
    i = 0
    for team in E8:
        bracket_df.at[i,'E8'] = team
        i+=8
    i = 0
    for team in FF:
        bracket_df.at[i,'FF'] = team
        i+=16
    i = 0
    for team in C:
        bracket_df.at[i,'C'] = team
        i+=32
    bracket_df.at[0,'W'] = winner
    
    return bracket_df

bracket_df = fill_bracket(bracket_df)

# pprint(bracket_df)

Our bracket prediction is made! Now let's export it to a csv file to store it.

In [9]:
formula1_2018_1_csv = bracket_df.to_csv (r'C:\Users\Brian\Documents\MarchMadness2019\formula1_2018_1.csv', index = None, header=True)

Let's make a few of these brackets just to get the hang of mass producing brackets.

In [10]:
for i in range(1,6):
    filename = r'C:\Users\Brian\Documents\MarchMadness2019\brackets\formula1_2018_' + str(i) + '.csv'
    bracket_df = fill_bracket(bracket_df)
    write_csv = bracket_df.to_csv (filename, index = None, header=True)
print('Successfully completed')

Successfully completed


Well, what are you waiting for? Check the path you saved your csv files to!

### That's it! Feel free to modify to your liking and generate as many brackets as you want to store! 